In [1]:
import pandas as pd								# data manipulation
import numpy as np								# graphs
import sys										# system functions
import warnings									# indicating warnings
warnings.filterwarnings('ignore')				# ignore by default
pd.set_option('display.max_columns', None)		# show all columns
pd.set_option('display.max_rows', None)			# show all rows
np.set_printoptions(threshold=sys.maxsize)		# show arrays full
pd.set_option("display.max_colwidth", -1)		# show a complete text in a cell
from xlsxwriter.workbook import Workbook		# CSV writing
import csv										# CSV writing	
import pymysql									# MySql connection
import datetime									# time conversions

# Question1: 
Can you sort a numerical list in Python?
This list contains strings not integers. So first, it needs to be converted 

In [2]:
mylist = ["5", "8", "1", "2", "10"]

In [3]:
# Convert to an int-containing list
mylist = [int(i) for i in mylist]

In [4]:
# Now we can sort it
mylist.sort()
mylist

[1, 2, 5, 8, 10]

# Question2: 
Write a code to count the number of capital letters in the drivers file.
First we should read the file

In [5]:
df = pd.read_csv('drivers_table.csv')

In [6]:
# Let's see what we have inside of it!
df.head(10)
# Sounds too much uppercases!

,id,driver,invoice_details,vehicle
0,1,ALBERTO,CARGO,Motorbike
1,2,SANTIAGO,GH,Van/Car
2,3,CAROLINA,COURIER-S,Van/Car
3,4,DIEGO,LOV,Van/Car
4,5,JOSE MANUEL,IV,Van/Car
5,6,RAFAEL,DONAD,Motorbike
6,7,CRISTINA,TEJA,Van/Car
7,8,CLARA,EXPRESS,Van/Car
8,9,AINHOA,MAR,Van/Car
9,10,PABLO,GILO,Van/Car


In [7]:
# We define categorical_feature as a list of features wth dtype of object
# Then we use for loop to see how many capital letters we have
# thanks to pandas functions such as findall, contains, ... we can easily find and then count the capital letters.
class color:
   BOLD = '\033[1m'
   END = '\033[0m'

categorical_features=[feature for feature in df.columns if df[feature].dtypes=='O']
for column in categorical_features:
    print('In column',column,'there are',color.BOLD, df[column].str.findall(r'[A-Z]').str.len().sum(),color.END,'capital letters.')

In column driver there are  704  capital letters.
In column invoice_details there are  494  capital letters.
In column vehicle there are  186  capital letters.


# Question3: 
Can you write a function to calculate the sum of a list of numbers?
Using for loop, we define the range which is from 0 to maximum the length of a list
and we extract all values of the list and write it one after another one in a pandas df, and then we sum it.

In [8]:
def MySum(mylist):
    dfcal = pd.DataFrame(data={'cal':['']}, columns=['cal'])
    for index in range(0,len(mylist)):
        dfcal['cal'][index] = mylist[index]
    return dfcal['cal'].sum()

In [9]:
# Let's test it!
mytest = [10,0,1,7,-3]

In [10]:
# It seems to be working fine
MySum(mytest)

15

# Question4: 
Write a function to count the total number of lines in the “drivers.csv” file.
Did you mean 'rows/indexes' by 'line'? if so. we can use shape

In [11]:
df.shape[0]

100

# Question5: 
Could you provide a code that executes the query you have created previously in question 6 of SQL and export the result to a CSV?
I would like to answer this question in 2 different approaches:

**A: Connect directly to a server:** I connect to my local MySQL server directly, use the same code more or less from SQL question6 and save the extracted table in the same folder as this notebook is. We can also directly connect to BigQuery using API and magic Ipython.

**B: Use Python codes:** I use python codes to answer the question6 of SQL section from scratch.

In [12]:
# A:
# MySql connector defined
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password='maryam35913', db='girrafe')

# I will copy and paste my query here. I just changed 'TIMESTAMPDIFF' a bit since that one functions slightly different in MySQL
Query="SELECT b.driver AS DriverName, round(COUNT(a.Attempted_time)/(TIMESTAMPDIFF(MINUTE,min(a.Delivery_Start),max(a.Delivery_End))/60),2)  AS ProductivityRate FROM orders_table a LEFT JOIN drivers_table b ON a.driver_id=b.id WHERE a.Order_Status = 'delivered' GROUP BY b.driver ORDER BY a.driver_id  ;"

cursor=conn.cursor()
cursor.execute(Query)
rows=cursor.fetchall()
column_names = [i[0] for i in cursor.description]
fp = open('Q6-ApproachA.csv', 'w')
myFile = csv.writer(fp, lineterminator = '\n')
myFile.writerow(column_names)   
myFile.writerows(rows)
fp.close()

# You can find the exported file in the same folder.

In [13]:
# Read order table, rename driver to column to make the merge easier, and merge them.
df2 = pd.read_csv('orders_table.csv')
df.rename(columns={'id':'driver_id'}, inplace=True)
dfmerge = df.merge(df2, on="driver_id")

In [14]:
# Deal with datetype and makes it ready for a subtraction
dfmerge['Delivery Start'] = pd.to_datetime(dfmerge['Delivery Start'])
dfmerge['Delivery End'] = pd.to_datetime(dfmerge['Delivery End'])

In [15]:
# Sorting time makes it easier to furthur analysis
dfmerge = dfmerge.sort_values(['Delivery Start', 'Delivery End'])

In [16]:
# B: Python codes.
dfresult = pd.DataFrame(data={'Driver':[''],'PR':['']}, columns=['Driver', 'PR'])
index = 0
for driver in dfmerge['driver'].unique():
    dfresult.at[index,'Driver'] = driver
    
    length = dfmerge.loc[dfmerge['driver'] == driver]['Delivery End'].iloc[-1] - dfmerge.loc[dfmerge['driver'] == driver]['Delivery Start'].iloc[0]
    dfresult.at[index,'PR'] = round((dfmerge.loc[dfmerge['driver'] == driver].shape[0]) / (length / np.timedelta64(1, 'h')),2)
    
    index= index + 1
    
dfresult.to_csv('Q6-ApproachB.csv', index=None)
dfresult

# Both approaches have yielded the identical numbers.

,Driver,PR
0,SANDRA,0.89
1,JOSE ANGEL,0.67
2,LUIS,1.25
3,JUAN JOSE,0.6
4,XAVIER,1
5,MARCOS,0.67
6,MARIA LUISA,0.75
7,EVA MARIA,0.33
8,MARIA ANGELES,1.25
9,JULIO,0.33
